# 0.0 Imports

In [1]:
import pandas as pd
import regex as re

# 0.1 Helper functions

In [2]:
df_raw_1 = pd.read_csv('../data/data-1703806461.csv')
df_raw_2 = pd.read_csv('../data/data-1703806894.csv')
df_raw_3 = pd.read_csv('../data/data-1703805541.csv')
df_raw_4 = pd.read_csv('../data/data-1703779777.csv')
df_raw_5 = pd.read_csv('../data/data-1703969930.csv')

df_raw = pd.concat([df_raw_1, df_raw_2, df_raw_3, df_raw_4, df_raw_5])
df_raw = df_raw.drop_duplicates(subset=['id'])

In [3]:
df_raw.shape

(22341, 12)

In [21]:
df_raw.sample(1).T

,44
Unnamed: 0,0
id,2657050159
usableAreas,['47']
createdAt,2023-09-21T14:07:18.919Z
displayAddressGeolocation,"{'lon': -46.562258, 'lat': -23.525512}"
parkingSpaces,[1]
suites,[]
bathrooms,[1]
bedrooms,[2]
pricingInfos,"[{'yearlyIptu': '100', 'price': '430000', 'bus..."


In [4]:
df0 = df_raw.copy()

In [5]:
df0 = df0.drop(columns=['Unnamed: 0'], axis=1)

In [62]:
df0.sample(1).T

,5933
id,2678852996
usableAreas,150
createdAt,2023-12-26T00:55:06.914Z
displayAddressGeolocation,"{'lon': -46.736066, 'lat': -23.628053}"
parkingSpaces,3
suites,1
bathrooms,2
bedrooms,3
pricingInfos,"[{'yearlyIptu': '48000', 'price': '1150000', '..."
amenities,"['BALCONY', 'NUMBER_OF_FLOORS', 'GOURMET_BALCO..."


In [7]:
df0['usableAreas'] = df0['usableAreas'].str.extract(r"(\d+)")
df0['parkingSpaces'] = df0['parkingSpaces'].str.extract(r"(\d+)")
df0['suites'] = df0['suites'].str.extract(r"(\d+)")
df0['bathrooms'] = df0['bathrooms'].str.extract(r"(\d+)")
df0['bedrooms'] = df0['bedrooms'].str.extract(r"(\d+)")

In [14]:
df0['createdAt_dt'] = pd.to_datetime(df0['createdAt'], format='ISO8601')

In [8]:
df0.isna().sum()

id                              0
usableAreas                     0
createdAt                       0
displayAddressGeolocation    1218
parkingSpaces                 629
suites                       2284
bathrooms                       0
bedrooms                        0
pricingInfos                    0
amenities                       0
unitFloor                       0
dtype: int64

In [60]:
df0[['locationLon', 'locationLat']] = df0['displayAddressGeolocation'].str.split(',', expand=True)

df0['locationLat'] = df0['locationLat'].str.replace("'lat': ", "").str.replace('}', '').astype(float)

df0['locationLon'] = df0['locationLon'].str.replace("{'lon': ", "").str.replace('{', '').astype(float)

In [67]:
df0[['yearlyIptu', 'price', 'businessType', 'monthlyCondoFee']] = df0['pricingInfos'].str.split(',', expand=True)

ValueError: Columns must be same length as key

In [89]:
df1 = df0.copy()

In [82]:
df1['pricingInfos']= df1['pricingInfos'].str.findall()

In [99]:
df1.iloc[0, 8]

"[{'yearlyIptu': '1800', 'price': '575000', 'businessType': 'SALE', 'monthlyCondoFee': '550'}]"

In [189]:
re.findall(r"(\d+)", "[('price', '3100'), ('price', '720000')]")

['3100', '720000']

In [203]:
df1['yearlyIptu'] = df1['pricingInfos'].str.findall(r"(yearlyIptu)': '(\d+)").astype(str).str.extract(r"(\d+)")

df1['price'] = df1['pricingInfos'].str.findall(r"(price)': '(\d+)").astype(str).str.findall(r"(\d+)").apply(lambda x: x[1] if len(x)>1 else x[0])

df1['monthlyCondoFee'] = df1['pricingInfos'].str.findall(r"(monthlyCondoFee)': '(\d+)").astype(str).str.extract(r"(\d+)")

#df1['yearlyIptu'] = df1['yearlyIptu'].str.extract(r"(\d+)")

In [182]:
df1.iloc[998,8]

"[{'rentalInfo': {'period': 'MONTHLY', 'warranties': [], 'monthlyRentalTotalPrice': '3895'}, 'yearlyIptu': '203', 'price': '3100', 'businessType': 'RENTAL', 'monthlyCondoFee': '795'}, {'yearlyIptu': '203', 'price': '720000', 'businessType': 'SALE', 'monthlyCondoFee': '795'}]"

In [205]:
df1.iloc[998,16]

'720000'

In [206]:
df1.sample(1).T

,5161
id,2679266895
usableAreas,96
createdAt,2023-12-27T16:43:25.122Z
displayAddressGeolocation,"{'lon': -46.711365, 'lat': -23.522746}"
parkingSpaces,3
suites,0
bathrooms,1
bedrooms,3
pricingInfos,"[{'yearlyIptu': '680', 'price': '1484400', 'bu..."
amenities,"['PLAYGROUND', 'PARTY_HALL', 'PETS_ALLOWED']"
